In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [0]:
from google.colab import files

import os
import sys
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras.constraints import MaxNorm

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger,ModelCheckpoint
from PIL import Image

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array


In [0]:
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving gcs.json to gcs.json
User uploaded file "gcs.json" with length 2297 bytes


In [0]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/content/gcs.json"

In [0]:
!ls -al /content/gcs.json

-rw-r--r-- 1 root root 2297 Mar 15 01:19 /content/gcs.json


In [0]:
def parse_func(s_example):
    features = {
                'label':tf.io.FixedLenFeature((44,),tf.float32),
                'image':tf.io.FixedLenFeature((128,128,1),tf.float32),
                }
    example = tf.io.parse_single_example(s_example, features=features)
    return example
def imgSerialization (image,label):
    example = tf.train.Example(features=tf.train.Features(feature={
        'label':_floats_feature(label),
        'image':_floats_feature(image.reshape(-1)),
        }))
    return example.SerializeToString()

def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

def array_overlay(B,b):
    B_center_r ,B_center_c = B.shape[0] // 2,B.shape[1] // 2
    H ,W,_ = np.minimum(b.shape,B.shape)
    row_start,column_start = B_center_r - H//2,B_center_c - W//2
    B[row_start:row_start + H,column_start:column_start + W,:] = b
    return B
    
def affine(img,theta = 0,left=0,up=0):
    x0 = img.size[0] // 2
    y0 = img.size[1] // 2
    a = np.cos(theta)
    b = - np.sin(theta)
    c =  x0 + y0*np.sin(theta) - x0*np.cos(theta) #left/right (i.e. 5/-5)
    d = np.sin(theta)
    e = np.cos(theta)
    f = y0 - x0*np.sin(theta) - y0*np.cos(theta) #up/down (i.e. 5/-5)
    img= img.transform(img.size, Image.AFFINE, (a, b, c, d, e, f))    
    return img.transform(img.size, Image.AFFINE, (1, 0, left, 0, 1, up))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
    #return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (_, _) = mnist.load_data()
x_train = (x_train / 255).astype(np.float32)
x_train = x_train.reshape((-1, 28,28,1))
new_data =[]
new_label= []
for _ in range(30000):
    B = np.zeros((128,128,1),dtype=np.float32)
    num = np.random.randint(4)
    indexes = np.random.randint(60000,size=num+1)
    b= np.concatenate(x_train[indexes],axis=1)

    theta = np.random.normal(0,np.pi/8)
    #theta = 0
    H,W,_ = B.shape 
    h,w,_ = b.shape
    H = H//2
    W=W//2
    left = np.random.uniform(np.abs(w * np.cos(theta) /2)-W, W - np.abs(w * np.cos(theta) /2))
    up = np.random.uniform(np.abs(w * np.sin(theta) /2)-H, H - np.abs(w * np.sin(theta) /2))
    image = array_to_img(array_overlay(B,b))
    new_img = affine(image,theta,up=up,left=left)
    new_data.append(img_to_array(new_img))
    ll= list(y_train[indexes])
    while (len(ll)<4):
        ll.append(0)
    a = np.concatenate([one_hot(np.array([x]),10) for x in ll])
    b = one_hot(np.array([num]),4)
    c = np.concatenate([a,b])
    new_label.append(c)
new_data = np.array(new_data)/255.0
new_label= np.array(new_label,dtype=np.float32)


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
data =[]
label =[]
for i in range(4):
    data.append(new_data[np.argmax(new_label[:,40:],axis=-1)==i])
    label.append(new_label[np.argmax(new_label[:,40:],axis=-1)==i])


In [0]:
for k in range(4):
  writer = tf.io.TFRecordWriter('gs://my-data-bucket-colab/digits/digits-{}.tfrecords'.format(k))
  for i in range(len(data[k])):
      s_example = imgSerialization(data[k][i],label[k][i])
      writer.write(s_example)
  writer.close()